У цьому ДЗ ми потренуємось розв'язувати задачу багатокласової класифікації за допомогою логістичної регресії з використанням стратегій One-vs-Rest та One-vs-One, оцінити якість моделей та порівняти стратегії.

### Опис задачі і даних

**Контекст**

В цьому ДЗ ми працюємо з даними про сегментацію клієнтів.

Сегментація клієнтів – це практика поділу бази клієнтів на групи індивідів, які схожі між собою за певними критеріями, що мають значення для маркетингу, такими як вік, стать, інтереси та звички у витратах.

Компанії, які використовують сегментацію клієнтів, виходять з того, що кожен клієнт є унікальним і що їхні маркетингові зусилля будуть більш ефективними, якщо вони орієнтуватимуться на конкретні, менші групи зі зверненнями, які ці споживачі вважатимуть доречними та які спонукатимуть їх до купівлі. Компанії також сподіваються отримати глибше розуміння уподобань та потреб своїх клієнтів з метою виявлення того, що кожен сегмент цінує найбільше, щоб точніше адаптувати маркетингові матеріали до цього сегменту.

**Зміст**.

Автомобільна компанія планує вийти на нові ринки зі своїми існуючими продуктами (P1, P2, P3, P4 і P5). Після інтенсивного маркетингового дослідження вони дійшли висновку, що поведінка нового ринку схожа на їхній існуючий ринок.

На своєму існуючому ринку команда з продажу класифікувала всіх клієнтів на 4 сегменти (A, B, C, D). Потім вони здійснювали сегментовані звернення та комунікацію з різними сегментами клієнтів. Ця стратегія працювала для них надзвичайно добре. Вони планують використати ту саму стратегію на нових ринках і визначили 2627 нових потенційних клієнтів.

Ви маєте допомогти менеджеру передбачити правильну групу для нових клієнтів.

В цьому ДЗ використовуємо дані `customer_segmentation_train.csv`[скачати дані](https://drive.google.com/file/d/1VU1y2EwaHkVfr5RZ1U4MPWjeflAusK3w/view?usp=sharing). Це `train.csv`з цього [змагання](https://www.kaggle.com/datasets/abisheksudarshan/customer-segmentation/data?select=train.csv)

**Завдання 1.** Завантажте та підготуйте датасет до аналізу. Виконайте обробку пропущених значень та необхідне кодування категоріальних ознак. Розбийте на тренувальну і тестувальну вибірку, де в тесті 20%. Памʼятаємо, що весь препроцесинг ліпше все ж тренувати на тренувальній вибірці і на тестувальній лише використовувати вже натреновані трансформери.
Але в даному випадку оскільки значень в категоріях небагато, можна зробити обробку і на оригінальних даних, а потім розбити - це простіше. Можна також реалізувати процесинг і тренування моделі з пайплайнами. Обирайте як вам зручніше.

In [ ]:
!pip install -U imbalanced-learn

In [21]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTENC
from imblearn.combine import SMOTETomek

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
df = pd.read_csv("/content/drive/MyDrive/Machine Learning/Data/customer_segmentation_train.csv")
df.head()

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,462809,Male,No,22,No,Healthcare,1.0,Low,4.0,Cat_4,D
1,462643,Female,Yes,38,Yes,Engineer,NaN,Average,3.0,Cat_4,A
2,466315,Female,Yes,67,Yes,Engineer,1.0,Low,1.0,Cat_6,B
3,461735,Male,Yes,67,Yes,Lawyer,0.0,High,2.0,Cat_6,B
4,462669,Female,Yes,40,Yes,Entertainment,NaN,High,6.0,Cat_6,A


Подивимось на відсотковий розподіл сегментів клієнтів:

In [62]:
df.Segmentation.value_counts(normalize=True)

Segmentation
D    0.281111
A    0.244422
C    0.244175
B    0.230293
Name: proportion, dtype: float64

In [9]:
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42, stratify = df.Segmentation)

In [10]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6454 entries, 917 to 2661
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               6454 non-null   int64  
 1   Gender           6454 non-null   object 
 2   Ever_Married     6343 non-null   object 
 3   Age              6454 non-null   int64  
 4   Graduated        6395 non-null   object 
 5   Profession       6348 non-null   object 
 6   Work_Experience  5808 non-null   float64
 7   Spending_Score   6454 non-null   object 
 8   Family_Size      6190 non-null   float64
 9   Var_1            6394 non-null   object 
 10  Segmentation     6454 non-null   object 
dtypes: float64(2), int64(2), object(7)
memory usage: 605.1+ KB


In [11]:
train_df.isnull().sum()

ID                   0
Gender               0
Ever_Married       111
Age                  0
Graduated           59
Profession         106
Work_Experience    646
Spending_Score       0
Family_Size        264
Var_1               60
Segmentation         0
dtype: int64

Бачимо, що 6 колонок (Ever_Married, Graduated, Profession, Work_Experience, Family_Size, Var_1) мають пропущені значення.

In [12]:
train_df[['Work_Experience',	'Family_Size']].describe()

,Work_Experience,Family_Size
count,5808.000000,6190.000000
mean,2.671488,2.839742
std,3.434175,1.531119
min,0.000000,1.000000
25%,0.000000,2.000000
50%,1.000000,3.000000
75%,4.000000,4.000000
max,14.000000,9.000000


In [13]:
train_df.Work_Experience.mode(), train_df.Family_Size.mode()

(0    1.0
 Name: Work_Experience, dtype: float64,
 0    2.0
 Name: Family_Size, dtype: float64)

Пропущені значення Work_Experience можемо замінити 1, яка є і модою і медіаною, це, на мою думку краще відображає тенденції в даних аніж середнє значення.

Пропущені значення Family_Size я би замінила на 3, яка є медіаною і середнім.

In [14]:
train_df.Ever_Married.value_counts(), train_df.Graduated.value_counts(),  train_df.Profession.value_counts()

(Ever_Married
 Yes    3734
 No     2609
 Name: count, dtype: int64,
 Graduated
 Yes    4007
 No     2388
 Name: count, dtype: int64,
 Profession
 Artist           2005
 Healthcare       1064
 Entertainment     745
 Doctor            567
 Engineer          562
 Lawyer            491
 Executive         489
 Marketing         230
 Homemaker         195
 Name: count, dtype: int64)

  Оскільки пропущених значень в стовпчиках Graduated і Ever_Married доволі мало, їх, на мою думку, можна замінити найбільш поширеним значенням.
  
  
  Для поля Profession я би створила нову групу Unknown.

  Що стосується поля Var_1 (Anonymised Category for the customer), яке імовірно має вплив на цільову змінну, я пропоную видалити рядки, для яких його значення не визначено.



In [15]:
# function to handle missing data in a dataframe
def fill_in_missing_data(df, train_df):
  df.dropna(subset=['Var_1'], inplace=True)
  df['Profession'].fillna(value='Unknown', inplace=True)
  df['Work_Experience'].fillna(value=train_df['Work_Experience'].median(), inplace=True)
  df['Family_Size'].fillna(value=train_df['Family_Size'].median(), inplace=True)
  df['Graduated'].fillna(value=train_df['Graduated'].mode()[0], inplace=True)
  df['Ever_Married'].fillna(value=train_df['Ever_Married'].mode()[0], inplace=True)

In [16]:
fill_in_missing_data(train_df, train_df)
fill_in_missing_data(val_df, train_df)

In [23]:
# split the data into inputs and targets
input_cols = list(train_df.columns)[1:-1]
target_col = 'Segmentation'
train_inputs, train_targets = train_df[input_cols].copy(), train_df[target_col].copy()
val_inputs, val_targets = val_df[input_cols].copy(), val_df[target_col].copy()

In [24]:
# define numeric and categorical columns
numeric_cols = train_inputs.select_dtypes(include=np.number).columns.tolist()
categorical_cols = train_inputs.select_dtypes('object').columns.tolist()
numeric_cols, categorical_cols

(['Age', 'Work_Experience', 'Family_Size'],
 ['Gender',
  'Ever_Married',
  'Graduated',
  'Profession',
  'Spending_Score',
  'Var_1'])

In [25]:
# encode categorical data
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
encoder.fit(train_inputs[categorical_cols])
encoded_cols = list(encoder.get_feature_names_out(categorical_cols))

train_inputs[encoded_cols] = encoder.transform(train_inputs[categorical_cols])
val_inputs[encoded_cols] = encoder.transform(val_inputs[categorical_cols])

In [26]:
# scale numeric data
scaler = MinMaxScaler()
scaler.fit(train_inputs[numeric_cols])
train_inputs[numeric_cols] = scaler.transform(train_inputs[numeric_cols])
val_inputs[numeric_cols] = scaler.transform(val_inputs[numeric_cols])

In [27]:
# define X and y
X_cols = numeric_cols + encoded_cols
X_train = train_inputs[X_cols]
X_val = val_inputs[X_cols]
y_train = train_targets
y_val = val_targets

**Завдання 2. Важливо уважно прочитати все формулювання цього завдання до кінця!**

Застосуйте методи ресемплингу даних SMOTE та SMOTE-Tomek з бібліотеки imbalanced-learn до тренувальної вибірки. В результаті у Вас має вийти 2 тренувальних набори: з апсемплингом зі SMOTE, та з ресамплингом з SMOTE-Tomek.

Увага! В нашому наборі даних є як категоріальні дані, так і звичайні числові. Базовий SMOTE не буде правильно працювати з категоріальними даними, але є його модифікація, яка буде. Тому в цього завдання є 2 виконання

  1. Застосувати SMOTE базовий лише на НЕкатегоріальних ознаках і порівняти лін регресію на апсампл даних без категоріальних ознак.

  2. Переглянути інформацію про метод [SMOTENC](https://imbalanced-learn.org/dev/references/generated/imblearn.over_sampling.SMOTENC.html#imblearn.over_sampling.SMOTENC) і використати цей метод в цій задачі. За цей спосіб буде +3 бали за це завдання і він рекомендований для виконання.

  **Підказка**: аби скористатись SMOTENC треба створити змінну, яка містить індекси ознак, які є категоріальними (їх номер серед колонок) і передати при ініціації екземпляра класу `SMOTENC(..., categorical_features=cat_feature_indeces)`.
  
  Ви також можете розглянути варіант використання варіації SMOTE, який працює ЛИШЕ з категоріальними ознаками [SMOTEN](https://imbalanced-learn.org/dev/references/generated/imblearn.over_sampling.SMOTEN.html)

In [28]:
#SMOTENC resampling
encoded_col_indices = [X_train.columns.get_loc(col) for col in encoded_cols]

smotenc = SMOTENC(categorical_features=encoded_col_indices, categorical_encoder=encoder, random_state=42)
X_train_smotenc, y_train_smotenc = smotenc.fit_resample(X_train, y_train)

In [29]:
y_train_smotenc.value_counts()

Segmentation
A    1797
B    1797
C    1797
D    1797
Name: count, dtype: int64

Перевіримо, чи категоріальні дані були вірно згенеровані (чи вони містять тільки 0 і 1):

In [39]:
for col in encoded_cols:
    print(col, X_train_smotenc[col].isin([0, 1]).all())

Gender_Female True
Gender_Male True
Ever_Married_No True
Ever_Married_Yes True
Graduated_No True
Graduated_Yes True
Profession_Artist True
Profession_Doctor True
Profession_Engineer True
Profession_Entertainment True
Profession_Executive True
Profession_Healthcare True
Profession_Homemaker True
Profession_Lawyer True
Profession_Marketing True
Profession_Unknown True
Spending_Score_Average True
Spending_Score_High True
Spending_Score_Low True
Var_1_Cat_1 True
Var_1_Cat_2 True
Var_1_Cat_3 True
Var_1_Cat_4 True
Var_1_Cat_5 True
Var_1_Cat_6 True
Var_1_Cat_7 True


In [47]:
# SMOTETomek resampling
smotetomek = SMOTETomek(smote=smotenc, random_state=42)
X_train_smotetomek, y_train_smotetomek = smotetomek.fit_resample(X_train, y_train)

In [49]:
y_train_smotetomek.value_counts()

Segmentation
C    1465
D    1414
B    1388
A    1339
Name: count, dtype: int64

In [50]:
for col in encoded_cols:
    print(col, X_train_smotetomek[col].isin([0, 1]).all())

Gender_Female True
Gender_Male True
Ever_Married_No True
Ever_Married_Yes True
Graduated_No True
Graduated_Yes True
Profession_Artist True
Profession_Doctor True
Profession_Engineer True
Profession_Entertainment True
Profession_Executive True
Profession_Healthcare True
Profession_Homemaker True
Profession_Lawyer True
Profession_Marketing True
Profession_Unknown True
Spending_Score_Average True
Spending_Score_High True
Spending_Score_Low True
Var_1_Cat_1 True
Var_1_Cat_2 True
Var_1_Cat_3 True
Var_1_Cat_4 True
Var_1_Cat_5 True
Var_1_Cat_6 True
Var_1_Cat_7 True


**Завдання 3**.
  1. Навчіть модель логістичної регресії з використанням стратегії One-vs-Rest з логістичною регресією на оригінальних даних, збалансованих з SMOTE, збалансованих з Smote-Tomek.  
  2. Виміряйте якість кожної з натренованих моделей використовуючи `sklearn.metrics.classification_report`.
  3. Напишіть, яку метрику ви обрали для порівняння моделей.
  4. Яка модель найкраща?
  5. Якщо немає суттєвої різниці між моделями - напишіть свою гіпотезу, чому?

In [63]:
# logistic regression with one-vs-rest (OvR) strategy
log_reg = LogisticRegression(solver='liblinear')
ovr_model = OneVsRestClassifier(log_reg)

In [64]:
# train the model on the original data
ovr_model.fit(X_train, y_train)
ovr_predictions = ovr_model.predict(X_val)

print(classification_report(y_val, ovr_predictions))

              precision    recall  f1-score   support

           A       0.43      0.46      0.44       390
           B       0.42      0.17      0.24       370
           C       0.49      0.64      0.56       390
           D       0.65      0.76      0.70       448

    accuracy                           0.52      1598
   macro avg       0.50      0.51      0.49      1598
weighted avg       0.50      0.52      0.50      1598



In [65]:
# train the model on the data resampled using SMOTENC
ovr_model.fit(X_train_smotenc, y_train_smotenc)
ovr_predictions_smotenc = ovr_model.predict(X_val)

print(classification_report(y_val, ovr_predictions_smotenc))

              precision    recall  f1-score   support

           A       0.43      0.46      0.45       390
           B       0.42      0.26      0.32       370
           C       0.51      0.62      0.56       390
           D       0.67      0.73      0.70       448

    accuracy                           0.53      1598
   macro avg       0.51      0.52      0.51      1598
weighted avg       0.52      0.53      0.52      1598



In [66]:
# train the model on the data resampled using SMOTE-Tomek
ovr_model.fit(X_train_smotetomek, y_train_smotetomek)
ovr_predictions_smotetomek = ovr_model.predict(X_val)

print(classification_report(y_val, ovr_predictions_smotetomek))

              precision    recall  f1-score   support

           A       0.43      0.48      0.46       390
           B       0.44      0.24      0.31       370
           C       0.50      0.63      0.56       390
           D       0.67      0.72      0.70       448

    accuracy                           0.53      1598
   macro avg       0.51      0.52      0.50      1598
weighted avg       0.52      0.53      0.51      1598



Висновки:

Для оцінки моделей можна використати як macro так і weighted average, оскільки вони дуже подібні (через несуттєвий дисбаланс класів) і видають послідовні результати. Серед протестованих моделей найкращий результат має модель навчена на даних із використанням SMOTENC ресамплінгу (macro avg = 0.51, weighted avg = 0.52).

Загалом результати є досить схожими для всіх трьох моделей, на мою думку, тому що дисбаланс класів був дуже помірним, і застосування технік ресамплінгу не призвело до суттєвого покращення. Хоча найменш представлений клас В ми таки навчились розпізнавати краще на resampled даних (його F1 score покращився з 0.24 до 0.32), тож я б застосовувала другу модель.